In [100]:
from netCDF4 import Dataset, num2date, date2num
import numpy as np
import datetime as dt
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import random
get_ipython().magic(u'matplotlib inline')

In [101]:
# ## Read CRU composite data
nc =Dataset ('/Users/kwesi/Downloads/combinations/mix_all/new/cru_tmp_mon_81-13_mix9.std.rc123A.nc', 'r')
for i in nc.variables:
    print(i,nc.variables[i].shape)
    

(u'time', (176,))
(u'lon', (80,))
(u'lat', (70,))
(u'anom', (176, 70, 80))


In [102]:
# Read variables and close the file
lon = nc.variables['lon'][:]
lat = nc.variables['lat'][:]
tempmean = nc.variables['anom'][:]
ntime = nc.variables['time'][:]
nc.close()

In [103]:
d = np.array(tempmean)
d[d>10000]=np.nan
#print(d[d>10000])
print d.shape

d1=d
ndat=d1.shape[0]
print d1.shape

(176, 70, 80)
(176, 70, 80)


In [104]:
iters = 10000
shape = [iters]
shape.extend(d1.shape[1:])
means = np.zeros(shape, dtype=np.float32)

for i in range(10000):
    indices = np.random.choice(ndat, ndat)
    means[i] = d1[indices,:].mean(axis=0)
    
print means.shape

(10000, 70, 80)


In [105]:
percentiles=np.nanpercentile(means, [5, 95], axis=0)
print percentiles.shape
lower=percentiles[0,:]
upper=percentiles[1,:]

(2, 70, 80)


In [106]:
#pctrange=upper-lower
#plt.plot(pctrange[40,:])
#print pctrange.shape

In [107]:
ncdataset=Dataset("cru_temp_90pctl_mix9a_CI.nc", "w", format='NETCDF4_CLASSIC')
ncdataset.createDimension('lon', len(lon))
ncdataset.createDimension('lat', len(lat))
#ncdataset.createDimension('time', 5600)

<type 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 70

In [108]:
longitudes = ncdataset.createVariable('lon',np.float32, ('lon',))
latitudes  = ncdataset.createVariable('lat',np.float32, ('lat',))
#time  = ncdataset.createVariable('time',np.float32, ('time',))

In [109]:
longitudes[:]=lon[:]
latitudes[:]=lat[:]
#time[:]=np.arange(1,5601)

In [110]:
var1=ncdataset.createVariable('lower',np.float32,('lat', 'lon',))
var2=ncdataset.createVariable('upper',np.float32,('lat', 'lon',))
var1[:,:]=lower
var2[:,:]=upper
ncdataset.close()